In [ ]:
import pandas
import requests
import spotipy
import string
import re

from spotipy.oauth2 import SpotifyClientCredentials
from genderize import Genderize

"""
# Not really working
# From 'Song for Bob Dylan', extracted the name 'Song' :(
import spacy
import spacy.cli
spacy.cli.download('en_core_web_lg')
en = spacy.load('en_core_web_lg')

"""


In [ ]:
with open('vars.env', 'r') as f:
    my_id = f.readline().rstrip('\n')
    my_secret = f.readline().rstrip('\n')
    playlistURI = f.readline().rstrip('\n')
    owner = f.readline().rstrip('\n')

client_credentials_manager = SpotifyClientCredentials(client_id=my_id, client_secret=my_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlistID = playlistURI[-22:]

In [ ]:
results = sp.user_playlist(owner, playlist_id=playlistID, fields='tracks, next')['tracks']

songs =  results['items']
while results['next']:
    results = sp.next(results)
    songs.extend(results['items'])

idSet = {track['added_by']['id'] for track in songs}    
us = {(id, sp.user(id)['display_name']) for id in idSet}
us = dict(us)
us['rmb101'] = 'rmb101'

print(len(songs))


In [ ]:
# Make the dataframe by pulling out the song name, artist and person who added it
table = [(track['track']['name'], track['track']['artists'][0]['name'], us[track['added_by']['id']]) 
         for track in songs]
df = pandas.DataFrame(table, columns = ['Song', 'Artist', 'Added by'])

# Get rid of anything after a hyphen, as this is all '2012 remastered' kinda text
# also remove all apostrophe s as this seems to confuse the names
df['Song'].replace(' - .*|\'s','', regex = True, inplace = True)

# set up regex to kill off remaining punctuation
punclist = string.punctuation
punc = re.compile('[%s]' % re.escape(punclist))

# bring in list of common words (ie not names) to remove
# if this works I might expand it
words = pandas.read_csv('top5000.csv')
wordlist = words['Top words'].tolist()


# function to remove the punctuation and common names
# returns as list because I'm going to run genderize over it as a list
def namesOnly(row):
    text = punc.sub('', row['Song'].lower()).split()
    newTitle = [word for word in text if word not in wordlist]
    return newTitle

df['ShortTitle'] = df.apply(namesOnly, axis = 1)


In [ ]:
# Function to try to assign a gender to each word in each shortened song title
def gender(song):
    response = Genderize().get(song['ShortTitle'])
    return response
    #return [(word['gender'], word['probability']) for word in response] 

df['GenderData'] = df.apply(gender, axis = 1)

df['GenderData'].head()
# I think speed this up by getting a single set of names to run it on first
# then join the results up with the list - so I don't do each name more than once

In [ ]:
# Make this part of main genderising function once happy with it
def tidy(song):
    words = [[word['gender'], word['probability']] for word in song['GenderData'] if word['gender']]
    guesses = [word[0] for word in words if word[1] > 0.7] 
    if len(set(guesses)) == 1:
        return guesses[0]
    elif not guesses:
        return ''
    else:
        return guesses

df['Guesses'] = df.apply(tidy, axis = 1)

df.head()

In [ ]:
df[['ShortTitle', 'Guesses']].loc[~df['Guesses'].astype(str).isin(['male', 'female']) ]

#df[['Song','Guesses']].to_csv('Attempt1.csv')
#dirty
#block
#rigby
#theme
#dear
#ill dance
#holland
#dont

In [ ]:
# Manually correct the ones it couldn't work out 

In [ ]:
# now for artists (argh)


# Not really working

def entity(text):
    nameList = [name for name in en(text['Song']).ents if name.label_ == 'PERSON']
    if nameList:
        return nameList[0]
    else:
        return None

df['SongNames'] = df.apply(entity, axis = 1)
df.head(50)